In [ ]:
import sys
sys.path.append(r"C:\Users\bonda\Documents\Bondan\TEWS Testing\skydrifter")
import skydrifter as sd
from skydrifter.Utils.partisan import *
from skydrifter.Utils.nonpartisan import *

In [ ]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import torch
import matplotlib.pyplot as plt
import pandas as pd
import seisbench.models as sbm
import seisbench
seisbench.use_backup_repository()

# Domestic Function

In [ ]:
def find_nan_with_data_index(tensor):
    nan_mask = torch.isnan(tensor)
    contains_nan = nan_mask.any().item()
    nan_indices = nan_mask.nonzero(as_tuple=True) if contains_nan else tuple()
    data_indices = nan_indices[0].unique().tolist() if contains_nan else []
    return data_indices

def remove_indices(lst, indices_to_remove):
    indices_to_remove = set(indices_to_remove)
    return [item for idx, item in enumerate(lst) if idx not in indices_to_remove]

# Data Cleaning

In [ ]:
input_folder = r"C:\Users\bonda\Documents\Bondan\TEWS Testing\dataset\2025 Januray Picking Dataset 100 Hz"

In [ ]:
metadata = pd.read_csv(join_by([input_folder,'metadata','dataset_metadata.csv'],separator='\\'))
metadata

In [ ]:
train_index = np.int64(metadata[metadata['splitting'] == 'train']['dataset_index'].tolist())
test_index = np.int64(metadata[metadata['splitting'] == 'test']['dataset_index'].tolist())

### 1. Cleaning Train Data

In [ ]:
real_train_index = copy.deepcopy(train_index)
real_train_index

In [ ]:
X_train = torch.zeros(len(train_index),3,3001)
y_train = torch.zeros(len(train_index),3,3001)
for i in range(0,len(train_index)):
    path_in_loop = join_by([input_folder,'data',str(train_index[i])+'.pth'],separator='\\')
    data_in_loop = torch.load(path_in_loop)
    X_train[i] = data_in_loop[0]
    path_in_loop = join_by([input_folder,'label',str(train_index[i])+'.pth'],separator='\\')
    label_in_loop = torch.load(path_in_loop)
    rearrange = torch.zeros(1,3,3001)
    rearrange[0,0] = label_in_loop[0,0]
    rearrange[0,1] = label_in_loop[0,1]
    rearrange[0,2] = label_in_loop[0,2]
    y_train[i] = rearrange[0]
    print(f"\rData Number {i+1} Loaded From {len(train_index)} Total Data Available | {(((i+1)/(len(train_index)))*100):.2f} % {''*200}",end=' ')

In [ ]:
data_indices = find_nan_with_data_index(X_train)
data_indices

In [ ]:
original_list = copy.deepcopy(real_train_index)
indices_to_remove = data_indices
train_index = remove_indices(original_list, indices_to_remove)

In [ ]:
X_train = torch.zeros(len(train_index),3,3001)
y_train = torch.zeros(len(train_index),3,3001)
for i in range(0,len(train_index)):
    path_in_loop = join_by([input_folder,'data',str(train_index[i])+'.pth'],separator='\\')
    data_in_loop = torch.load(path_in_loop)
    X_train[i] = data_in_loop[0]
    path_in_loop = join_by([input_folder,'label',str(train_index[i])+'.pth'],separator='\\')
    label_in_loop = torch.load(path_in_loop)

    rearrange = torch.zeros(1,3,3001)
    rearrange[0,0] = label_in_loop[0,0]
    rearrange[0,1] = label_in_loop[0,1]
    rearrange[0,2] = label_in_loop[0,2]
    y_train[i] = rearrange[0]
    
    print(f"\rData Number {i+1} Loaded From {len(train_index)} Total Data Available | {(((i+1)/(len(train_index)))*100):.2f} % {''*200}",end=' ')

In [ ]:
data_indices = find_nan_with_data_index(X_train)
data_indices

In [ ]:
X_train.shape

### 2. Cleaning Test Data

In [ ]:
real_test_index = copy.deepcopy(test_index)
real_test_index

In [ ]:
X_test = torch.zeros(len(test_index),3,3001)
y_test = torch.zeros(len(test_index),3,3001)
for i in range(0,len(test_index)):
    path_in_loop = join_by([input_folder,'data',str(test_index[i])+'.pth'],separator='\\')
    data_in_loop = torch.load(path_in_loop)
    X_test[i] = data_in_loop[0]
    path_in_loop = join_by([input_folder,'label',str(test_index[i])+'.pth'],separator='\\')
    label_in_loop = torch.load(path_in_loop)
    rearrange = torch.zeros(1,3,3001)
    rearrange[0,0] = label_in_loop[0,0]
    rearrange[0,1] = label_in_loop[0,1]
    rearrange[0,2] = label_in_loop[0,2]
    y_test[i] = rearrange[0]
    print(f"\rData Number {i+1} Loaded From {len(test_index)} Total Data Available | {(((i+1)/(len(test_index)))*100):.2f} % {''*200}",end=' ')

In [ ]:
data_indices = find_nan_with_data_index(X_test)
data_indices

In [ ]:
original_list = copy.deepcopy(real_test_index)
indices_to_remove = data_indices
test_index = remove_indices(original_list, indices_to_remove)

In [ ]:
X_test = torch.zeros(len(test_index),3,3001)
y_test = torch.zeros(len(test_index),3,3001)
for i in range(0,len(test_index)):
    path_in_loop = join_by([input_folder,'data',str(test_index[i])+'.pth'],separator='\\')
    data_in_loop = torch.load(path_in_loop)
    X_test[i] = data_in_loop[0]
    path_in_loop = join_by([input_folder,'label',str(test_index[i])+'.pth'],separator='\\')
    label_in_loop = torch.load(path_in_loop)

    rearrange = torch.zeros(1,3,3001)
    rearrange[0,0] = label_in_loop[0,0]
    rearrange[0,1] = label_in_loop[0,1]
    rearrange[0,2] = label_in_loop[0,2]
    
    y_test[i] = rearrange[0]
    print(f"\rData Number {i+1} Loaded From {len(test_index)} Total Data Available | {(((i+1)/(len(test_index)))*100):.2f} % {''*200}",end=' ')

In [ ]:
data_indices = find_nan_with_data_index(X_test)
data_indices

### 3. Make New Metadata Because NaN

In [ ]:
train_metadata = []
for i in range(0,len(train_index)):
    train_in_loop = metadata[metadata['dataset_index'] == train_index[i]].iloc[0]
    train_metadata.append(train_in_loop)
train_metadata = pd.DataFrame(train_metadata)
train_metadata.index = [i for i in range(0,len(train_metadata))]
print('data = ',X_train.shape)
print('label = ',y_train.shape)
print('metadata = ',len(train_metadata))
train_metadata

In [ ]:
test_metadata = []
for i in range(0,len(test_index)):
    test_in_loop = metadata[metadata['dataset_index'] == test_index[i]].iloc[0]
    test_metadata.append(test_in_loop)
test_metadata = pd.DataFrame(test_metadata)
test_metadata.index = [i for i in range(0,len(test_metadata))]
print('data = ',X_test.shape)
print('label = ',y_test.shape)
print('metadata = ',len(test_metadata))
test_metadata

In [ ]:
cleaned_metadata = pd.concat([train_metadata,test_metadata])
print(len(train_metadata)+len(test_metadata))
cleaned_metadata

In [ ]:
cleaned_metadata.to_csv('cleaned_metadata.csv',index=False)

# Get Descriptive Statistic of Each Component

In [ ]:
n = X_train.shape[0]
E = []
for i in range(0,n):
    E += X_train[i][0].numpy().tolist()

In [ ]:
n = X_train.shape[0]
N = []
for i in range(0,n):
    N += X_train[i][1].numpy().tolist()

In [ ]:
n = X_train.shape[0]
Z = []
for i in range(0,n):
    Z += X_train[i][2].numpy().tolist()

In [ ]:
component = pd.DataFrame()
component['E'] = E
component['N'] = N
component['Z'] = Z

In [ ]:
summary = component.describe()
print(summary)

In [ ]:
summary.to_csv('clenaed_metadata_summary.csv',index=False)

# Determine Order

In [ ]:
client = Client("GFZ")
t = UTCDateTime("2007/01/02 05:48:50")
stream = client.get_waveforms(network="CX", station="PB01", location="*", channel="HH?", starttime=t-100, endtime=t+100)
inv = client.get_stations(level='response',network='CX',station='PB01')
stream.remove_response(inventory=inv)
stream.detrend(type='demean')
fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(111)
for i in range(3):
    ax.plot(stream[i].times(), stream[i].data, label=stream[i].stats.channel)
ax.legend()

In [ ]:
stn = stream.copy()
waveform_duration = stn[0].stats.endtime-stn[0].stats.starttime
stn.resample(sampling_rate=(3001/waveform_duration))
print(stn)
print(stn.plot())

In [ ]:
model_original = sbm.PhaseNet()
sb_pt_model = model_original.from_pretrained('original')
model_original.load_state_dict(sb_pt_model.state_dict())

In [ ]:
model_original.labels,model_original.component_order

In [ ]:
original = {
    'data': {0:'E',1:'N',2:'Z'},
    'label': {0:'P',1:'S',2:'N'},
}
# target = {
#     'data': {0:model_original.component_order[0],1:model_original.component_order[1],2:model_original.component_order[2]},
#     'label': {0:model_original.labels[0],1:model_original.labels[1],2:model_original.labels[2]},
# }
print('original')
print(original)
print('target')
# print(target)

In [ ]:
import torch
X = torch.zeros(1,3,3001)
X[0,0] = torch.FloatTensor(stn.select(component='Z')[0].data)
X[0,1] = torch.FloatTensor(stn.select(component='N')[0].data)
X[0,2] = torch.FloatTensor(stn.select(component='E')[0].data)
min_vals = X.min(dim=-1, keepdim=True).values  
max_vals = X.max(dim=-1, keepdim=True).values 
X = 2 * (X - min_vals) / (max_vals - min_vals) - 1

In [ ]:
with torch.no_grad():
    model_original.eval()
    y_pred = model_original(X)
fig, axs = plt.subplots(2,figsize=(17,10))
axs[0].plot(X[0,0].numpy(),label='0')
axs[0].plot(X[0,1].numpy(),label='1')
axs[0].plot(X[0,2].numpy(),label='2')
axs[0].set_xlabel('Data Point/Time')
axs[0].set_ylabel('Amplitude')
axs[0].set_title('Waveform Data')
axs[0].legend()
axs[1].plot(y_pred[0,0].numpy(),label='0')
axs[1].plot(y_pred[0,1].numpy(),label='1')
axs[1].plot(y_pred[0,2].numpy(),label='2')
axs[1].set_xlabel('Data Point/Time')
axs[1].set_ylabel('Probability')
axs[1].set_title('Prediction')
axs[1].legend()
fig.tight_layout()

In [ ]:
np.save('original_order.npy',original)
# np.save('target_order.npy',target)